In [41]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

scaler = StandardScaler()

df = pd.read_csv("../CONFIGS_FAMILIA_SCORE_FINAl.csv", sep=";", encoding="utf-8", decimal=",")

df.columns = ['familia', 'eluyente1', 'eluyente2', 'columna_nombre',
       'columna_usp_code', 'columna_length', 'columna_id',
       'columna_particle_size', 'columna_temperature', 'columna_flowrate',
       'tiempo_muerto', 'score']

categorical_cols = ["familia", "eluyente1", "eluyente2", "columna_nombre", "columna_usp_code"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Convertimos todo a string para evitar errores
    label_encoders[col] = le  # Guardamos los encoders para futuras conversiones

# Rellenamos valores nulos con la mediana
df.fillna(df.median(), inplace=True)

Q1 = df['columna_length'].quantile(0.25)
Q3 = df['columna_length'].quantile(0.75)
IQR = Q3 - Q1

df_cleaned = df[(df['columna_length'] >= Q1 - 1.5 * IQR) & (df['columna_length'] <= Q3 + 1.5 * IQR)]

# Aplicamos log-transform a la variable objetivo
df_cleaned['score'] = np.log1p(df_cleaned['score'])  # Aplicamos log(1 + x) para evitar problemas con ceros

# Escalamos las características
X = df_cleaned.drop(columns=["score"])
y = df_cleaned["score"]

X_scaled = scaler.fit_transform(X)

# Dividimos en conjunto de entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor()

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RMSE del modelo: {rmse}")



RMSE del modelo: 12.308335709470448


C:\Users\cmora\AppData\Local\Temp\ipykernel_40224\4253678715.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['score'] = np.log1p(df_cleaned['score'])  # Aplica log(1 + x) para evitar problemas con ceros


In [42]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df = pd.read_csv("../CONFIGS_FAMILIA_SCORE_FINAl.csv", sep=";", encoding="utf-8", decimal=",")

df.columns = ['familia', 'eluyente1', 'eluyente2', 'columna_nombre',
       'columna_usp_code', 'columna_length', 'columna_id',
       'columna_particle_size', 'columna_temperature', 'columna_flowrate',
       'tiempo_muerto', 'score']

categorical_cols = ["familia", "eluyente1", "eluyente2", "columna_nombre", "columna_usp_code"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Convertimos todo a string para evitar errores
    label_encoders[col] = le  # Guardamos los encoders para futuras conversiones

df.fillna(df.median(), inplace=True)  # Rellena NaN con la mediana

Q1 = df['columna_length'].quantile(0.25)
Q3 = df['columna_length'].quantile(0.75)
IQR = Q3 - Q1

df_cleaned = df[(df['columna_length'] >= Q1 - 1.5 * IQR) & (df['columna_length'] <= Q3 + 1.5 * IQR)]

# Aplica log(1 + x) de manera segura para evitar SettingWithCopyWarning
df_cleaned.loc[:, 'score'] = np.log1p(df_cleaned['score'])  

df_scaled = scaler.fit_transform(df_cleaned)

X = df_cleaned.drop(columns=["score"])
y = df_cleaned["score"]

# Dividimos en conjunto de entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creación y entrenamiento del modelo XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
rmse = np.sqrt(np.mean((y_pred - y_test) ** 2))

print(f"RMSE del modelo: {rmse}")


RMSE del modelo: 12.40889194029859


In [48]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error

df = pd.read_csv("../CONFIGS_FAMILIA_SCORE_FINAl.csv", sep=";", encoding="utf-8", decimal=",")

df.columns = ['familia', 'eluyente1', 'eluyente2', 'columna_nombre',
              'columna_usp_code', 'columna_length', 'columna_id',
              'columna_particle_size', 'columna_temperature', 'columna_flowrate',
              'tiempo_muerto', 'score']

categorical_cols = ["familia", "eluyente1", "eluyente2", "columna_nombre", "columna_usp_code"]

ordinal_encoders = {}
for col in categorical_cols:
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    df[col] = encoder.fit_transform(df[[col]])  # Ajustar y transformar los valores
    ordinal_encoders[col] = encoder  # Guardamos los encoders para más tarde

# Rellenar los valores faltantes con la mediana de la columna
df.fillna(df.median(), inplace=True)

# Escalar los datos
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop(columns=["score"]))  # No escalar la columna "score"

# Aplicar log(1 + x) a la variable objetivo para evitar problemas con ceros
df['score'] = np.log1p(df['score'])

X = df.drop(columns=["score"])
y = df["score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE del modelo: {rmse}")

RMSE del modelo: 13.15934503237291


In [49]:


nueva_config = pd.DataFrame({
    "familia": ["Aryltetralin lignans (CHEMONTID:0003425)"],
    "eluyente1": ["eluent.1.h2o 0"],
    "eluyente2": ["eluent.2.acn 0"],
    "columna_nombre": ["Agilent ZORBAX Extend-C18"],
    "columna_usp_code": ["column.usp.code_L1"],
    "columna_length": [500.0],
    "columna_id": [21.0],
    "columna_particle_size": [18.0],
    "columna_temperature": [350.0],
    "columna_flowrate": [1.0],
    "tiempo_muerto": [11025.0]
})

# Transformar las columnas categóricas de la nueva configuración
for col in ["familia", "eluyente1", "eluyente2", "columna_nombre", "columna_usp_code"]:
    nueva_config[col] = ordinal_encoders[col].transform(nueva_config[[col]])

# Escalar la nueva configuración
nueva_config_scaled = scaler.transform(nueva_config)

# Predecir el score para la nueva configuración
predicted_score = xgb_model.predict(nueva_config_scaled)

print(f"Score predicho para la nueva configuración: {predicted_score[0]}")


Score predicho para la nueva configuración: 21.39250946044922


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error

df = pd.read_csv("../CONFIGS_FAMILIA_SCORE_FINAl.csv", sep=";", encoding="utf-8", decimal=",")

df.columns = ['familia', 'eluyente1', 'eluyente2', 'columna_nombre',
              'columna_usp_code', 'columna_length', 'columna_id',
              'columna_particle_size', 'columna_temperature', 'columna_flowrate',
              'tiempo_muerto', 'score']

categorical_cols = ["familia", "eluyente1", "eluyente2", "columna_nombre", "columna_usp_code"]

ordinal_encoders = {}
for col in categorical_cols:
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    df[col] = encoder.fit_transform(df[[col]])
    ordinal_encoders[col] = encoder  # Guardamos los encoders para usarlos en predicciones

# Rellenar valores faltantes con la mediana
df.fillna(df.median(), inplace=True)

# Escalar las características numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop(columns=["score"]))  # No escalar la variable objetivo

df['score'] = np.log1p(df['score'])

X = df.drop(columns=["score"])
y = df["score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE del modelo: {rmse}")


RMSE del modelo: 13.563457753373319


In [4]:

nueva_config = pd.DataFrame({
    "familia": ["Aryltetralin lignans (CHEMONTID:0003425)"],
    "eluyente1": ["eluent.1.h2o 0"],
    "eluyente2": ["eluent.2.acn 0"],
    "columna_nombre": ["Agilent ZORBAX Extend-C18"],
    "columna_usp_code": ["column.usp.code_L1"],
    "columna_length": [500.0],
    "columna_id": [21.0],
    "columna_particle_size": [18.0],
    "columna_temperature": [350.0],
    "columna_flowrate": [1.0],
    "tiempo_muerto": [11025.0]
})

# Transformar las columnas categóricas de la nueva configuración
for col in ["familia", "eluyente1", "eluyente2", "columna_nombre", "columna_usp_code"]:
    nueva_config[col] = ordinal_encoders[col].transform(nueva_config[[col]])

# Escalar la nueva configuración
nueva_config_scaled = scaler.transform(nueva_config)

# Predecir el score para la nueva configuración
predicted_score = rf_model.predict(nueva_config_scaled)

print(f"Score predicho para la nueva configuración: {predicted_score[0]}")


Score predicho para la nueva configuración: 14.434649572449041


c:\Users\cmora\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error

df = pd.read_csv("../CONFIGS_FAMILIA_SCORE_FINAl.csv", sep=";", encoding="utf-8", decimal=",")

df.columns = ['familia', 'eluyente1', 'eluyente2', 'columna_nombre',
              'columna_usp_code', 'columna_length', 'columna_id',
              'columna_particle_size', 'columna_temperature', 'columna_flowrate',
              'tiempo_muerto', 'score']

categorical_cols = ["familia", "eluyente1", "eluyente2", "columna_nombre", "columna_usp_code"]

ordinal_encoders = {}
for col in categorical_cols:
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    df[col] = encoder.fit_transform(df[[col]])  # Ajustar y transformar los valores
    ordinal_encoders[col] = encoder  # Guardamos los encoders para más tarde

df.fillna(df.median(), inplace=True)

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop(columns=["score"]))  # No escalar la columna "score"

# Aplicar log(1 + x) a la variable objetivo para evitar problemas con ceros
df['score'] = np.log1p(df['score'])

X = df.drop(columns=["score"])
y = df["score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape de las entradas para la CNN (convertir en 2D)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1, 1)  # Imagen 1D
X_test_reshaped = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1, 1)

model = models.Sequential()

# Capa Conv2D: Aplicando un filtro a las características
model.add(layers.Conv2D(64, (2, 1), activation='relu', input_shape=(X_train_reshaped.shape[1], 1, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 1)))

# Capa Conv2D adicional para aprender más características
model.add(layers.Conv2D(128, (2, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 1)))

# Aplanar la salida de la capa convolucional para conectarla a la red densa
model.add(layers.Flatten())

# Capa densa con unidades de salida
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1))  # Como es un problema de regresión, no usaremos activación aquí

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

history = model.fit(X_train_reshaped, y_train, epochs=20, batch_size=32, validation_data=(X_test_reshaped, y_test))

y_pred = model.predict(X_test_reshaped)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE del modelo CNN: {rmse}")


c:\Users\cmora\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 10, 1, 64)      │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 5, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 4, 1, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 2, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,729 (194.25 KB)

 Trainable params: 49,729 (194.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 10/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 5m

ValueError: Input contains NaN.